<h1> Structured Machine Learning using Tensorflow, Google Cloud Datalab and Cloud ML</h1>
<hr />
<b>This notebook demonstrates a process to deploy a ML model to CloudML. It leverages a pre-built machine learning model to predict Length of Stay in ED and inpatient care settings. Finally it runs an inference job on the CloudML Engine to render predictions. This is step 2 of 2.</b>
<h3>
<br />
<ol>
<li> Setup Environment </li> <br />
<li> Deploy and Run a ML Model on CloudML </li>
</ol></h3>
<hr />

<h2> 1. Setup Environment</h2>
<ul>
    <li>Initialize environment variables for your environment</li>
    <li>Please change the values of the following before executing rest of the cells in this notebook: <br />
        <b>1. GCP_PROJECT and </b> <br />
        <b>2. GCS_BUCKET </b> <br />
        <b>3. GCS_REGION </b>
    </li>
</ul>

In [17]:
import os
GCP_PROJECT = 'dp-workspace'
GCS_BUCKET = 'gs://cluster19-bkt'
GCS_REGION = 'us-central1'
os.putenv("REGION", GCS_REGION)
TF_RECORD_SEQEX = GCS_BUCKET+'/synthea/serv/seqex*'
os.putenv("SEQEX_IN_GCS", TF_RECORD_SEQEX)
MODEL_PATH = GCS_BUCKET+'/synthea/model/'
os.putenv("MODEL_IN_GCS", MODEL_PATH+"*")
SAVED_MODEL_PATH = MODEL_PATH + 'export'
os.putenv("SAVED_MODEL_IN_GCS", SAVED_MODEL_PATH+"*")
SERVING_DATASET = GCS_BUCKET+'/synthea/serv/seqex-00002-of-00003.tfrecords'
os.putenv("SERVING_DATASET", SERVING_DATASET)
INFERENCE_PATH = MODEL_PATH + 'infer'
os.putenv("INFERENCE_PATH", INFERENCE_PATH)
os.putenv("MODEL_NAME", "tf_fhir_los")

<b>Import dependencies. </b>

In [2]:
# from apache_beam.options.pipeline_options import PipelineOptions
# from apache_beam.options.pipeline_options import GoogleCloudOptions
# from apache_beam.options.pipeline_options import StandardOptions
# import apache_beam as beam
from tensorflow.core.example import example_pb2
import tensorflow as tf
import time

from proto import version_config_pb2
from proto.stu3 import google_extensions_pb2
from proto.stu3 import resources_pb2

from google.protobuf import text_format
from py.google.fhir.labels import label
from py.google.fhir.labels import bundle_to_label
from py.google.fhir.seqex import bundle_to_seqex
from py.google.fhir.models import model
from py.google.fhir.models.model import make_estimator

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
/usr/local/envs/py2env/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _ni_label
/usr/local/envs

<b>Optionally, enable logging for debugging.</b>

In [3]:
import logging
logger = logging.getLogger()
#logger.setLevel(logging.INFO)
logger.setLevel(logging.ERROR)

<b> Previous step saved Sequence Examples into GCS. Let's examine file size and location of the Sequence Examples we will use of the inference. </b>

In [4]:
%bash
gsutil ls -l ${SEQEX_IN_GCS}

  45296041  2019-03-06T22:22:06Z  gs://cluster19-bkt/synthea/serv/seqex-00002-of-00003.tfrecords
TOTAL: 1 objects, 45296041 bytes (43.2 MiB)


<h2> 2. Deploy and Run ML Model on Cloud ML</h2>
<ul>
    <li>A pre-trained ML Model which was exported to GCS in step 1 will be deployed to Cloud ML Serving.</li>
</ul>
<b>2a. Let's start with exporting our model for serving.<b>

In [9]:
from py.google.fhir.models.model import get_serving_input_fn
hparams = model.create_hparams()
time_crossed_features = [
        cross.split(':') for cross in hparams.time_crossed_features if cross
    ]
LABEL_VALUES = ['less_or_equal_3', '3_7', '7_14', 'above_14']
estimator = make_estimator(hparams, LABEL_VALUES, MODEL_PATH)
serving_input_fn = get_serving_input_fn(hparams.dedup, hparams.time_windows, hparams.include_age, hparams.categorical_context_features, hparams.sequence_features, time_crossed_features)
export_dir = estimator.export_savedmodel(SAVED_MODEL_PATH, serving_input_fn)
os.putenv("MODEL_BINARY", export_dir)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 180, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba7c855850>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_model_dir': 'gs://cluster19-bkt/synthea/model/', '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_environment': 'local', '_save_summary_steps': 100}


I0306 22:31:04.616381 140440319678208 tf_logging.py:115] Using config: {'_save_checkpoints_secs': 180, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba7c855850>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_model_dir': 'gs://cluster19-bkt/synthea/model/', '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_environment': 'local', '_save_summary_steps': 100}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 180, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba7c8554d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_model_dir': 'gs://cluster19-bkt/synthea/model/', '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_environment': 'local', '_save_summary_steps': 100}


I0306 22:31:04.621371 140440319678208 tf_logging.py:115] Using config: {'_save_checkpoints_secs': 180, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba7c8554d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_model_dir': 'gs://cluster19-bkt/synthea/model/', '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_environment': 'local', '_save_summary_steps': 100}


INFO:tensorflow:Calling model_fn.


...

I0306 22:31:14.258912 140440319678208 tf_logging.py:115] Assets added to graph.


INFO:tensorflow:No assets to write.


I0306 22:31:14.263230 140440319678208 tf_logging.py:115] No assets to write.


INFO:tensorflow:SavedModel written to: gs://cluster19-bkt/synthea/model/export/temp-1551911464/saved_model.pb


I0306 22:31:23.941159 140440319678208 tf_logging.py:115] SavedModel written to: gs://cluster19-bkt/synthea/model/export/temp-1551911464/saved_model.pb


<b>2b. List all the models deployed currently in the Cloud ML Engine</b>

In [25]:
%%bash
gcloud ml-engine models list

Listed 0 items.


<b>2c. Optionally run following cell to delete previously deployed model. </b>

In [23]:
%%bash
gcloud ml-engine versions delete v1 --model ${MODEL_NAME} -q
gcloud ml-engine models delete $MODEL_NAME -q

Deleting version [v1]......
..............................done.
Deleting model [tf_fhir_los]...
done.


<b>2d. Run following cell to create a new model if it does not exist </b>

In [26]:
%%bash
gcloud ml-engine models create $MODEL_NAME --regions=$REGION

Created ml engine model [projects/dp-workspace/models/tf_fhir_los].


<b> 2e. List versions of the Model</b>

In [14]:
%%bash
gcloud ml-engine versions list --model ${MODEL_NAME}

Listed 0 items.


<b> 2f. Run following cell to create a new version of the model. Increment the version number like v1, v2, v3 </b> <br />
Optionally, you can delete a version using: <br />
gcloud ml-engine versions delete v1 --model ${MODEL_NAME} -q

In [27]:
%%bash
#gcloud ml-engine versions delete v1 --model ${MODEL_NAME} -q
gcloud ml-engine versions create v1 \
    --model ${MODEL_NAME} \
    --origin ${MODEL_BINARY} \
    --runtime-version 1.12

Creating version (this might take a few minutes)......
.............................................................................................................................................................................................done.


<b> 2g. Run an inference job on CloudML engine </b>

In [18]:
%%bash
INFER_JOB_NAME="job_inf_$(date +%Y%m%d_%H%M%S)"
gcloud ml-engine jobs submit prediction $INFER_JOB_NAME --model $MODEL_NAME     --version v1     --data-format tf-record     --region $REGION     --input-paths $SERVING_DATASET     --output-path $INFERENCE_PATH


jobId: job_inf_20190306_223914
state: QUEUED


Job [job_inf_20190306_223914] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe job_inf_20190306_223914

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs job_inf_20190306_223914


<b>You can check the status of the job and other information on <a href="https://pantheon.corp.google.com/mlengine/jobs">GCP CloudML page</a> </b>

<b> 2h. View the prediction (output) generated by the inference job </b>

In [21]:
%%bash
gsutil cat ${INFERENCE_PATH}/prediction.results-00000-of-00001

{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.46091485023498535, 0.1920824497938156, 0.1735013723373413, 0.1735013723373413]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.33346351981163025, 0.2637665867805481, 0.20138496160507202, 0.20138496160507202]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.43521541357040405, 0.20636126399040222, 0.17921166121959686, 0.17921166121959686]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.41660845279693604, 0.20461316406726837, 0.1893891543149948, 0.1893891543149948]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.3496762216091156, 0.2616179883480072, 0.1943529099225998, 0.1943529099225998]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.4166741967201233, 0.235443577170372, 0.17394115030765533, 0.17394115030765533]}
{"classes": ["less_or_equal_3", "3_7", "7_14", "above_14"], "scores": [0.3644

<b>You can check the status of the job and other information on <a href="https://pantheon.corp.google.com/mlengine/jobs">GCP CloudML page</a> </b>